In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_input(path):
    """ Reads input file from passed path and returns as numpy array. No
    preprocessing is done. """
    with open(path, 'r') as f:
        data = np.array([l.strip() for l in f.readlines()])
    return data

### Day 1

#### Part 1

In [4]:
day_1_data = read_input("day_1/input.txt").astype(int)

In [5]:
def calc_positive_diff(arr):
    """ Given an array, count how many 'next values' are higher then the previous value. """
    return sum((arr[1:] - arr[:-1]) > 0)


def sum_by_window(arr, window_size=3):
    """ Slides a window_size sized window over an array and calculates the sums of the windows. """
    return pd.Series(arr).rolling(window=window_size).sum().dropna().values

In [6]:
calc_positive_diff(day_1_data)

1692

#### Part 2

In [7]:
# Calculate diff using sum of sliding window
calc_positive_diff(sum_by_window(day_1_data, window_size=3))

1724

### Day 2

#### Part 1

In [8]:
day_2_data = read_input("day_2/input.txt")

# Split lines into command and values and converts the values into integers
day_2_data = [[l[0], int(l[1])] for l in [line.split() for line in day_2_data]]

In [9]:
x_pos = 0
y_pos = 0

for command, value in day_2_data:
    if command.startswith('f'):
        x_pos += value
    elif command.startswith('d'):
        y_pos += value
    elif command.startswith('u'):
        y_pos -= value
        
x_pos * y_pos

1250395

#### Part 2

In [10]:
x_pos = 0
y_pos = 0
aim = 0

for command, value in day_2_data:
    if command.startswith('f'):
        x_pos += value
        y_pos += value * aim
    elif command.startswith('d'):
        aim += value
    elif command.startswith('u'):
        aim -= value
        
x_pos * y_pos

1451210346

In [11]:
# Numpy solution part 1
day_2_data = read_input("day_2/input.txt")
commands, values = zip(*[line.split() for line in day_2_data])
# Turn into arrays for slicing possibilities
commands = np.array(commands)
values = np.array(values).astype(int)

x_pos = values[commands == 'forward'].sum()
y_pos = values[commands == 'down'].sum() - values[commands == 'up'].sum()
x_pos * y_pos

1250395

### Day 3

In [24]:
def load_day_3_data():
    day_3_data = read_input("day_3/input.txt")

    # Turn into numpy array, every bit on its own
    day_3_data = np.array([[int(bit) for bit in bytes_] for bytes_ in day_3_data])
    
    return day_3_data

#### Part 1

In [25]:
day_3_data = load_day_3_data()

In [26]:
# Gamma is most commit bit per 'column'
gamma = np.array([np.argmax(np.bincount(day_3_data[:, i])) for i in range(day_3_data.shape[1])])

# Epsilon is least common bit per column, so the opposite of the gamma
epsilon = 1 - np.array(gamma)

In [27]:
def byte_array_to_int(arr):
    return int(''.join(arr.astype(str)), base=2)

In [28]:
# Transform binary to number
gamma = byte_array_to_int(gamma)
epsilon = byte_array_to_int(epsilon)

In [29]:
# Answer is gamma times epsilon
gamma * epsilon

1458194

#### Part 2

In [30]:
day_3_data = load_day_3_data()

In [31]:
def day_3_part_2(data, filter_by='max'):
    assert filter_by in ['min', 'max']
    
    if filter_by == 'min':
        default = 0
        min_max_func = np.argmin
    else:
        default = 1
        min_max_func = np.argmax
    
    for idx in range(data.shape[1]):
        count = np.bincount(data[:, idx])
        if len(set(count)) > 1 or len(set(data[:, idx])) == 1:
            mcv = min_max_func(count)
        # Value if column contains the same amount of both values
        else:
            mcv = default

        # Remove rows where value of current index is not the most common value
        data = data[data[:, idx] == mcv]
        if data.shape[0] <= 1:
            break
            
    return data

In [34]:
oxygen_rating = byte_array_to_int(day_3_part_2(day_3_data, filter_by='max')[0])
co2_rating = byte_array_to_int(day_3_part_2(day_3_data, filter_by='min')[0])

In [35]:
oxygen_rating * co2_rating

2829354

### Day 4

#### Part 1

In [141]:
with open('day_4/input.txt', 'r') as f:
    input_data = f.readlines()
    # Split comma separated string of numbers to list and parse all
    # string numbers to ints
    numbers = [int(i) for i in input_data[0].strip().split(',')]
    # The rest of the data are the bingo cards
    bingo_cards = [line.strip().split() for line in input_data[2:] if line != '\n']  # Remove empty lines
    
    # Using reshape get a 3d array of N 5 by 5 bingo cards
    bingo_card_numbers = np.array(bingo_cards).reshape(-1, 5, 5).astype(int)

In [152]:
class BingoCard:
    
    def __init__(self, values):
        self.values = values  # 2D array
        self.marked = np.zeros(shape=self.values.shape)  # Keep track of numbers that we marked
        self.card_size = self.values.shape[0]  # Assumes square card
        self.latest_value = None  # Needed for calculating the final answer
        
    def mark_number(self, value):
        self.latest_value = value
        # Mark a 1 on the indexes where we have a hit
        if value in self.values:
            self.marked[np.where(self.values==value)] = 1
            
    def check_for_bingo(self):
        # If the sum of a row/column is the same as the size of the card we have a bingo
        # Diagonals are not counted so this is easy
        return self.card_size in self.marked.sum(axis=0) or self.card_size in self.marked.sum(axis=1)
    
    def calculate_answer(self):
        # Sum of unmarked values * latest called value
        return self.values[self.marked == 0].sum() * self.latest_value
    
    def calculate_n_moves_until_win(self, bingo_numbers):
        """ Needed for part 2 """
        for n, number in enumerate(bingo_numbers):
            self.mark_number(number)
            if self.check_for_bingo():
                return n, self.calculate_answer()

In [146]:
bingo_cards = [BingoCard(card) for card in bingo_card_numbers]

for number in numbers:
    # Fill number on card
    # Dont assign to variable as we're updating class instances
    [card.mark_number(number) for card in bingo_cards]
    
    # Check for bingos
    bingo_checks = [card.check_for_bingo() for card in bingo_cards]
    
    # If we have bingo there will be a true in this list, we check this with any
    if any(bingo_checks):
        # Winning card is the index of True
        winning_card = bingo_cards[bingo_checks.index(True)]
        break
        
winning_card.calculate_answer()

38913

#### Part 2

In [148]:
bingo_cards = [BingoCard(card) for card in bingo_card_numbers]

# Get list of tuples containing amount of moves until win, and the score of the card
moves_til_win = [card.calculate_n_moves_until_win(numbers) for card in bingo_cards]

In [151]:
# Sorting ascendingly on n moves and taking the 1nd index we have our answer
sorted(moves_til_win, key=lambda tup: tup[0])[-1][1]

16836

### Day 5

#### Part 1

In [74]:
coordinates = read_input('day_5/input.txt')

# Coordinates is array of [[x1, y1], [x2, y2]] pairs
coordinates = np.array([[l.strip().split(',') for l in line.split('->')] for line in coordinates]).astype(int)

# Only keep coordinates where x1 == x2 or y1 == y2
hor_ver_coordinates = coordinates[(coordinates[:, 0, 0] == coordinates[:, 1, 0]) |
                                  (coordinates[:, 0, 1] == coordinates[:, 1, 1])]

In [75]:
def generate_empty_map(coordinates):
    # Using coordinates calculate max map size
    max_x = coordinates[:, :, 0].max()+1
    max_y = coordinates[:, :, 1].max()+1
    # Initialize empty map with all zeros
    return np.zeros(shape=(max_y, max_x))

In [76]:
map_ = generate_empty_map(hor_ver_coordinates)

In [77]:
for (x1, y1), (x2, y2) in hor_ver_coordinates:
    if x1 == x2 or y1 == y2:
        # Draw lines in the map by adding one
        # Using min and max we prevent slicing from high to low value, numpy
        # returns an empty array when we do that
        map_[min(y1, y2): max(y1, y2)+1, min(x1, x2): max(x1, x2)+1] += 1

In [78]:
# By counting all places where the value is higher than one we get our answer
len(np.where(map_ > 1)[0])

5280

#### Part 2

In [79]:
map_ = generate_empty_map(coordinates)

for (x1, y1), (x2, y2) in coordinates:
    # Horizontal/vertical line
    if x1 == x2 or y1 == y2:
        # Draw lines in the map by adding one
        # Using min and max we prevent slicing from high to low value, numpy
        # returns an empty array when we do that
        map_[min(y1, y2): max(y1, y2)+1, min(x1, x2): max(x1, x2)+1] += 1
    # Diagonal line
    else:
        # Cannot use the min/max trick for both x and y coordinates as this will result
        # in a flipped diagonal. We can simply fix this by using a negative stepsize, which
        # will result in a reversed range, and therefore no flipped lines
        x_stepsize = 1 if x2 > x1 else -1
        y_stepsize = 1 if y2 > y1 else -1

        # Because we also used to add 1 to the highest x and y, we need to substract
        # one from the lowest x and y if we have a reversed range, otherwise the lines
        # will be one too short
        x2 = x2 - 1 if x2 < x1 else x2 + 1
        y2 = y2 - 1 if y2 < y1 else y2 + 1
        for x, y in zip(range(x1, x2, x_stepsize), range(y1, y2, y_stepsize)):
            map_[y][x] += 1

In [80]:
len(np.where(map_ > 1)[0])

16716

### Day 6

#### Part 1

In [24]:
lanternfish_timers = read_input('day_6/input.txt')[0].split(',')
lanternfish_timers = np.array(lanternfish_timers).astype(int)

In [27]:
from tqdm import tqdm

n_days = 80

def lanternfish_model(original_population, n_days):
    """ Given an initial population and an amount of days, returns the
    amount of lanternfish after n_days days. """
    original_population = np.array([i for i in original_population])
    for day in range(n_days):
        # Subtract 1 from every timer
        original_population -= 1

        # Calculate amount of new lanternfishes by counting the amount of -1 timers
        zero_timers = np.where(original_population == -1)[0]

        if zero_timers.size > 0:
            # Add new lanternfish with timer 8 for every original timer that hits -1
            original_population = np.concatenate((original_population, np.full(len(zero_timers), 8)))

            # Reset -1 fishes to 6
            original_population[zero_timers] = 6
    
    return original_population.size

In [29]:
lanternfish_model(lanternfish_timers, 80)

354564

#### Part 2

In [43]:
# Above method grows exponentially so it doesnt work for large amount of days
# New method uses a list of counts where the index is the timer of the fishes
counts = [list(lanternfish_timers).count(i) for i in range(9)]

for _ in range(256):
    # Shift all items in count list to the left, the first element becomes
    # the last element (fishes with timer 8)
    counts = counts[1:] + [counts[0]]
    # Because the original timer 0 fishes dont die, we have to
    # add their count to index 6, because that is their timer after
    # multiplying
    counts[6] += counts[-1]

sum(counts)


1609058859115

### Day 7

#### Part 1

In [73]:
crab_positions = read_input("day_7/input.txt")[0].split(',')
crab_positions = np.array(crab_positions).astype(int)

In [74]:
def calculate_constant_fuel_cost(crab_pos, target):
    return sum(abs(crab_positions - target))

In [75]:
# Brute force as the data is not THAT big
costs = [(target, calculate_constant_fuel_cost(crab_positions, target)) for target in range(max(crab_positions))]
sorted(costs, key=lambda tup: tup[1])[0][1]

336120

#### Part 2

In [85]:

def calculate_linear_fuel_cost(crab_pos, target):
    """ Uses triangle numbers to calculate the costs """
    return sum((abs(crab_positions - target) * (abs(crab_positions - target) + 1)) / 2)


In [87]:
# Brute force still works fast enough
costs = [(target, calculate_linear_fuel_cost(crab_positions, target)) for target in range(max(crab_positions))]
sorted(costs, key=lambda tup: tup[1])[0][1]

96864235.0

### Day 8

#### Part 1

In [107]:
signals = read_input("day_8/input.txt")
signals = [line.split('|') for line in signals]

In [108]:
outputs = [s[1] for s in signals]

# Simply count the amount a token of length 2/3/4/7 occurs
sum([len(s) in [2, 3, 4, 7] for s in ' '.join(outputs).split(' ')])

488

#### Part 2

In [111]:
signals = [''.join(s) for s in signals]

Logic from https://imgur.com/a/LIS2zZr

- len(2) == 1
- len(4) == 4
- len(3) == 7
- len(7) == 8
- len(5) == [2, 3, 5]
    - contains(7) == 3
    - len(insersection(4) == 3) == 5
    - else 2
- len(6) == [0, 6, 9]
    - contains(4) == 9
    - contains(7) == 0
    - else 6

In [182]:
def token_to_charset(token):
    return set([c for c in token])

def charset_by_length(line, length):
    """ Used for getting set of the characters in codes for 1, 4, 7, and 8 """
    return set([[c for c in code] for code in line if len(code) == length][0])

def decode_line(line):
    # Dictionary foor looking up a number by length (for the numbers 1, 4, 7, 8)
    len_dict = {2: 1, 3: 7, 4: 4, 7: 8}

    # Dictionary with sets of characters for known numbers 1, 4, 7, and 8, using the
    # keys and values from len_dict
    char_dict = {k: charset_by_length(line, l) for l, k in len_dict.items()}

    decoded_tokens = []

    # Decoding using a for loop
    for token in line:
        charset = token_to_charset(token)
        # Simple lookup by length
        if len(charset) in [2, 3, 4, 7]:
            decoded_tokens.append(len_dict[len(charset)])
        # Length of 5 means 2, 3 or 5
        elif len(charset) == 5:
            # If the token contains all characters used by 7
            # the token is a three
            if char_dict[7].issubset(charset):
                decoded_tokens.append(3)
            # If the intersection of the characters is of size
            # 3 the token is a 5
            elif len(char_dict[4].intersection(charset)) == 3:
                decoded_tokens.append(5)
            # Only possible value is a 2
            else:
                decoded_tokens.append(2)
        # Length of 6 means 9, 0, or 6
        elif len(charset) == 6:
            # If the token contains all characters used by 4, the
            # token is a nine
            if char_dict[4].issubset(charset):
                decoded_tokens.append(9)
            # Same trick as above for the characters of 7, the token
            # is a 0
            elif char_dict[7].issubset(charset):
                decoded_tokens.append(0)
            # Final option a 6
            else:
                decoded_tokens.append(6)

    # Only return the final 4 values as integer, the rest is not needed for solution
    return int(''.join([str(i) for i in decoded_tokens[-4:]]))

In [184]:
sum([decode_line(line.split()) for line in signals])

1040429